# Importing Necessary Libraries

In this section, we'll import all the necessary libraries that our program will use.

In [ ]:
import os
from dotenv import load_dotenv
import pinecone
from langchain.chains import LLMChain
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema import HumanMessage, SystemMessage

# Loading Environment Variables

Here, we'll load environment variables from a `.env` file. This file typically contains sensitive information like API keys.

In [ ]:
# Load environment variables from .env file
load_dotenv()

# Get OpenAI, Pinecone API keys from environment variables
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
PINECONE_ENV = os.environ.get("PINECONE_ENV")

# Connecting to Pinecone

We'll initialize the Pinecone connection using the API key and environment details.

In [ ]:
# Connect to Pinecone
pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_ENV)

# Defining the Index Creation Function

This function checks if an index exists in Pinecone. If it does and `force_recreate` is set to `True`, it deletes and recreates the index. Otherwise, it simply loads the existing index.

In [ ]:
def create_or_load_index(index_name, force_recreate=False):
    if force_recreate and index_name in pinecone.list_indexes():
        # If we need to force recreate, and the index already exists, delete it
        pinecone.delete_index(index_name)

    if index_name not in pinecone.list_indexes():
        # Now we create the index if it doesn't exist (or if we just deleted it)
        pinecone.create_index(
            name=index_name,
            metric='cosine',
            dimension=1536
        )
        # Load and parse data
        loader = DirectoryLoader(
            './data', glob="**/*.txt", loader_cls=TextLoader,
            loader_kwargs={'autodetect_encoding': True, 'encoding': 'utf-8'}
        )
        documents = loader.load()

        # Split messages into chunks
        text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
        docs = text_splitter.split_documents(documents)

        # Initialize Pinecone vector store
        vectorstore = Pinecone.from_documents(
            docs, embeddings, index_name=index_name)
    else:
        # Initialize Pinecone vector store
        vectorstore = Pinecone.from_existing_index(index_name, embeddings)

    return vectorstore

# Using the Index Creation Function

We'll now call the function to either create or load the Pinecone index.

In [ ]:
index_name = "whatsapp-demo"
vectorstore = create_or_load_index(index_name, force_recreate=False)

# Querying the Vector Store

We'll use a sample query to search for similar content in our vector store.

In [ ]:
query = "How you doing today?"  # Replace with your query
result = vectorstore.similarity_search(query)

# Setting up the Chat Model and Prompt

In this section, we'll set up the chat model and define the prompt for generating conversational responses.

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=.7)

prompt_msgs = [
    SystemMessage(
        content="You are an AI assistant trying to simulate a conversation as Jose Cardama."
    ),
    HumanMessage(
        content="You are given the following parts taken from a conversation. Provide a conversational response based on what Jose Cardama typically responds to:"
    ),
    HumanMessagePromptTemplate.from_template("{context}"),
    HumanMessage(content="Tip: Do not answer more than what is asked."),
    HumanMessage(content="Tip: If you don't know the answer, you can reply with 'Hmm, I'm not sure.'. Don't try to make up an answer."),
    HumanMessage(
        content="Message:"
    ),
    HumanMessagePromptTemplate.from_template("{input}"),
]
prompt = ChatPromptTemplate(
    messages=prompt_msgs, input_variables=["context", "input"]
)
prompt.format(
    context=result[0].page_content,
    input=query,
)

chain = LLMChain(llm=llm, prompt=prompt, verbose=True)

# Running the Chain Model

Finally, we'll run the chain model to generate a response based on our query and the context from the vector store.

In [ ]:
print(chain.run({"context": result[0].page_content, "input": query}))